In [0]:
%auto
typeset_mode(True, display=False)

In [0]:
# One-time pad (will use this later)
def encrypt_xor(plain, key):
    n = min(len(plain), len(key))
    ascii_codes = [ord(plain[i]) ^^ ord(key[i]) for i in range(n)]
    return ''.join([chr(x) for x in ascii_codes])

In [0]:
%html
<p>From last time</p>
<p>3. Let's make our own random number generator: Let $a_0$ a seven-digit number (seed). Then generate $a_{n+1}$ by taking the final 7 digits of $a_n^2$. Let our random sequence at position $n$ be 0 or 1 depending on whether $a_n$'s third digit is even or odd. Is this a good random function (how to check)? Write a function that computes the period length depending on the seed.</p>

In [0]:
# Problem 3: to 
def third_dig(n):
    return floor(n%100000 / 10000)

In [0]:
(1234567 % 100000) / 10000.0

In [0]:
# Third digit of 25: seven-digits 0000025

In [0]:
third_dig(1234567)

In [0]:
third_dig(1101111)

In [0]:
third_dig(1234)

In [0]:
# Parameters: seed, length
def rand_seq(seed, N):
    ret = []
    last = seed
    for i in range(N):
        ret.append(third_dig(last) % 2)
        last = (last^2) % 10^7
    return ret

In [0]:
rand_seq(1234567, 20)

In [0]:
rand_seq(5, 20)

In [0]:
def period_of_rand_seq(seed):
    last = seed
    L = []
    N = 0
    while last not in L:
        L.append(last)
        N += 1
        last = (last^2) % 10^7
        #print last
    return N

In [0]:
period_of_rand_seq(1234567)

In [0]:
period_of_rand_seq(1)

In [0]:
[period_of_rand_seq(K) for K in [1..10]]

In [0]:
period_of_rand_seq(5)

In [0]:
2890625^2

In [0]:
5127657^2

In [0]:
[period_of_rand_seq(K) for K in [91..100]]

In [0]:
# Period not long enough -> not 'one-time'. Information leaked.

In [0]:
%html
<h2>Goal: look into indistinguishability</h2>
<p>"Game":&nbsp; two plaintexts and a single ciphertext are given. Guess which plainext the ciphertext corresponds to. If guesses siginificantly better than 50%: we are in trouble.</p>

In [0]:
# Example 1. 
# Plaintexts: "AAAAAAAA", "BBBBBBBB" (or longer versions)
# Method: XOR with pseudorandom function's output
# Key: uniformly random integer between 1 and 10000
# (Pseudo)randomness: take powers of seed, Output 0 if last digit: 0, 1, 2, 3, 4, output 1 if last digit is 5, 6, 7, 8, 9.

In [0]:
[2^k%10 for k in [1..10]]

In [0]:
# E.g.
def rand_gen_ex1(seed, N):
    return [0 if (seed^k)%10 <5 else 1     for k in range(N)]

In [0]:
rand_gen_ex1(2, 20)

In [0]:
rand_gen_ex1(11, 20)

In [0]:
rand_gen_ex1(1010, 20)

In [0]:
def convert_binary_list_to_str(L):
    ret = ""
    for i in range(floor(len(L) / 8)):
        ret += chr( sum([2^k * L[8*i+k] for k in range(8)] ) )
    return ret

In [0]:
ord("A")

In [0]:
65.digits(2)

In [0]:
L = 65.digits(2) + [0]

In [0]:
convert_binary_list_to_str(L)

In [0]:
def output_random_ciphertext_ex1():
    m0 = "AAAAAAAA"
    m1 = "BBBBBBBB"
    plain = m0 if randint(0,1) == 0 else m1
    seed = randint(1, 10000)
    L = rand_gen_ex1(seed, len(plain)*8)
    k = convert_binary_list_to_str(L)
    return encrypt_xor(plain, k), plain

In [0]:
output_random_ciphertext_ex1()

In [0]:
output_random_ciphertext_ex1()

In [0]:
output_random_ciphertext_ex1()

In [0]:
# Strategy 1: if AAAAAAAA or BBBBBBBB then I know for sure, otherwise guess
def tipp(s):
    L = ["AAAAAAAA", "BBBBBBBB"]
    if s in L:
        return s
    else:
        return L[randint(0,1)]

In [0]:
def check_success_ratio(method, N):
    ret = 0
    for i in range(N):
        c, m = output_random_ciphertext_ex1()
        s = method(c)
        if s == m:
            ret += 1
    return ret

In [0]:
check_success_ratio(tipp, 1000)

In [0]:
# brute force decrypt, try all keys
def tipp2(s):
    seed = 0
    L = ["AAAAAAAA", "BBBBBBBB"]
    finished = False
    
    while not finished:
        L2 = rand_gen_ex1(seed, 64)
        k = convert_binary_list_to_str(L2)
        m = encrypt_xor(s, k)
        if m in L:
            finished = True
        seed += 1
    return m

In [0]:
check_success_ratio(tipp2, 1000)

In [0]:
# Example 2.
# Plaintexts: "AAAAAAAA", "BBBBBBBB" (or longer versions)
# Method: XOR with pseudorandom function's output
# Key: uniformly random integer between 1 and 10000
# (Pseudo)randomness: take powers of seed, Output 0 if _FIRST_ digit: 0, 1, 2, 3, 4, output 1 if _FIRST_ digit is 5, 6, 7, 8, 9.

In [0]:
def rand_gen_ex2(seed, N):
    return [0 if str((seed^k))[0] in '01234' else 1     for k in range(N)]

In [0]:
rand_gen_ex2(18, 20)

In [0]:
[18^k for k in range(20)]

In [0]:
def output_random_ciphertext_general(method):
    m0 = "AAAAAAAA"
    m1 = "BBBBBBBB"
    plain = m0 if randint(0,1) == 0 else m1
    seed = randint(1, 10000)
    L = method(seed, len(plain)*8)
    k = convert_binary_list_to_str(L)
    return encrypt_xor(plain, k), plain

In [0]:
output_random_ciphertext_general(rand_gen_ex2)

In [0]:
def check_success_ratio(method_rand_gen, method_guess, N=100):
    ret = 0
    for i in range(N):
        c, m = output_random_ciphertext_general(method_rand_gen)
        s = method_guess(c)
        if s == m:
            ret += 1
    return ret

In [0]:
check_success_ratio(rand_gen_ex1, tipp, 1000)

In [0]:
def guess_ex2(s):
    seed = 0
    L = ["AAAAAAAA", "BBBBBBBB"]
    finished = False
    
    while not finished:
        L2 = rand_gen_ex2(seed, 64)
        k = convert_binary_list_to_str(L2)
        m = encrypt_xor(s, k)
        if m in L:
            finished = True
        seed += 1
    return m

In [0]:
check_success_ratio(rand_gen_ex2, guess_ex2, 100)

In [0]:
# attempt 2:
# First look at the 0-1 statistic of the output of psrandom seq. at specific position:
def stats_pos(method, pos):
    cnt = 0
    for k in range(10000):
        L = method(k, pos+1)
        if L[pos]:
            cnt += 1
    return cnt

In [0]:
stats_pos(rand_gen_ex2, 8)

In [0]:
stats_pos(rand_gen_ex2, 20)

In [0]:
# "AAAAAAAA" -> binary: [1, 0, 0, 0, 0, 0, 1, 0]*8
# "BBBBBBBB" -> binary: [0, 1, 0, 0, 0, 0, 1, 0]*8
def guess_ex2_v2(s):
    L = ["BBBBBBBB", "AAAAAAAA"]
    return L[ ord(s[3]) % 2]

In [0]:
check_success_ratio(rand_gen_ex2, guess_ex2_v2, 10000)